In [1]:
T = 24  # Planning horizon
for current_time in range(total_operation_period):
    # Receive new demand profile for the next T time steps
    demand_profile = get_new_demand_profile(current_time, T)


N =10 # number of nodes

#Define Decision Variables
C = cp.Variable((N, T))      # Charging power to BESS
D = cp.Variable((N, T))      # Discharging power from BESS
S = cp.Variable((N, T+1))    # Energy Level of BESS

P_diesel = cp.Variable((N, T))  # Diesel generation power
#....

#Parameters
alpha_C, alpha_D = 0.95, 0.95  # Efficiencies
S_max, S_min = 100, 0          # Energy levels in kWh
RC_max, RD_max = 10, 10        # Ramping rates in kW
#...

#BESS Dynamics
for t in range(1, T+1):    # Dynamics over time horizon T
    S[:, t] = S[:, t-1] + alpha_C * C[:, t-1] - D[:, t-1] / alpha_D

#Constraints
constraints = []

for t in range(T):
    constraints += [S_min <= S[:, t], S[:, t] <= S_max]
    constraints += [cp.abs(C[:, t] - C[:, t-1]) <= RC_max,
                    cp.abs(D[:, t] - D[:, t-1]) <= RD_max]

for t in range(T):
    constraints.append(cp.sum(P_diesel[:, t] + P_renewable[:, t] + P_BESS[:, t]) == demand_profile[t])
#...

#Objective Function
objective = cp.Minimize(cp.sum(P_diesel))

#Solution
problem = cp.Problem(objective, constraints)
problem.solve()



NameError: name 'total_operation_period' is not defined

In [ ]:
import cvxpy as cp
import numpy as np

def get_new_demand_profile(current_time, T):
    # Function for fetching demand profile over time horizon T from data
    end
def get_new_forecast_profile(current_time, T):
    # Function for fetching RES forecast profile over time horizon T from data
    end

def main():
    N = 10  # Number of nodes
    T = 24  # Planning horizon
    total_operation_period = 168  # Total operation period 
    
    # Parameters
    alpha_C, alpha_D = 0.95, 0.95  # Efficiencies
    S_max, S_min = 100, 0          # Energy levels in kWh
    RC_max, RD_max = 10, 10        # Ramping rates in kW
    #...
    
    
    initial_energy = np.random.rand(N) * S_max  # Initial energy level of BESS
    
    for current_time in range(total_operation_period):
        
        demand_profile = get_new_demand_profile(current_time, T)
        
        #  Decision Variables
        C = cp.Variable((N, T))      # Charging power to BESS
        D = cp.Variable((N, T))      # Discharging power from BESS
        S = cp.Variable((N, T+1))    # Energy Level of BESS
        P_diesel = cp.Variable((N, T))  # Diesel generation power
        #...
        
        # P_renewable forecasted for time horizon
        P_renewable =get_new_forecast_profile(current_time, T)
        
        # Initial BESS energy Level
        S[:, 0] = initial_energy
        
        # BESS Dynamics
        for t in range(1, T+1):
            S[:, t] = S[:, t-1] + alpha_C * C[:, t-1] - D[:, t-1] / alpha_D
        
        # Constraints
        constraints = []
        for t in range(T):
            constraints += [S_min <= S[:, t], S[:, t] <= S_max]
            constraints += [cp.abs(C[:, t] - C[:, t-1]) <= RC_max,
                            cp.abs(D[:, t] - D[:, t-1]) <= RD_max]
        #...
        
        # Demand fulfillment constraint
        for t in range(T):
            constraints.append(cp.sum(P_diesel[:, t] + P_renewable[:, t] - C[:, t] + D[:, t]) == demand_profile[t])
        
        # Objective Function: Minimize the total diesel generation
        objective = cp.Minimize(cp.sum(P_diesel))
        
        # Solve the problem
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        # Update for next iteration
        initial_SoC = S[:, 1].value  # Update initial SoC for the next iteration
        
        # Solution for the first time step
        if current_time == 0:
            print("Optimal diesel generation for the first hour of the operation period:")
            print(P_diesel.value[:, 0])

if __name__ == "__main__":
    main()
